# This is to convert from nmea to Plane Rectangular. Coordinate System by GSI(Geospatial Information Authority of Japan) HP.

# 緯度経度(nmea)から平面直角座標へ国土地理院のHPで変換したい場合

https://vldb.gsi.go.jp/sokuchi/surveycalc/surveycalc/bl2xyf.html

when calculate again,

if you want

%reset

In [1]:
#if you want
#check current directory
%pwd

'C:\\Users\\survey\\Dropbox\\My PC (SURVEY-C20)\\Documents\\GitHub\\GNSS'

In [1]:
#define working directroy
# %cd C:\\Users\\survey\\Desktop\\nmea20210705
%cd C:\Users\survey\Dropbox\My PC (SURVEY-C20)\Documents\GitHub\GNSS

C:\Users\survey\Dropbox\My PC (SURVEY-C20)\Documents\GitHub\GNSS


In [30]:
#check files
%ls

 ドライブ C のボリューム ラベルがありません。
 ボリューム シリアル番号は 3C86-6F6B です

 C:\Users\survey\Dropbox\My PC (SURVEY-C20)\Documents\GitHub\GNSS のディレクトリ

2021/07/09  20:08    <DIR>          .
2021/07/09  19:44    <DIR>          ..
2021/07/09  19:17               195 .gitignore
2021/07/09  20:06    <DIR>          .idea
2021/07/09  20:08            13,944 gnss.ipynb
2021/07/09  20:00                46 README.md
2021/07/09  18:49            60,812 VRS直_2021-07-05-12-48-07.rtk
               4 個のファイル              74,997 バイト
               3 個のディレクトリ  1,048,534,614,016 バイトの空き領域


In [4]:
import pandas as pd
import numpy as np
import csv

# 1. まずは，入力ファイルを設定します.

In [5]:
# inp_file="2021-07-05_10.nmea"
inp_file="C:\\Users\survey\Desktop\\nmea20210705\\2021-07-05_09+10_GNRMC.csv"
# inp_file="2021070509_XY.csv"
# # inp_file="VRS直_2021-07-05-12-48-07.rtk"
# inp_file="VRS直_2021-06-27-11-45-43.rtk"

# base_nameを取得しておきます．
# 出力ファイル名を自動で作成するため

In [6]:
type(inp_file)
base_name=inp_file.split('.',1)[0]
base_name
# inp_file="VRS直_2021-07-05-12-48-07_get.rtk"

'C:\\Users\\survey\\Desktop\\nmea20210705\\2021-07-05_09+10_GNRMC'

In [8]:
%pwd

'C:\\Users\\survey\\Dropbox\\My PC (SURVEY-C20)\\Documents\\GitHub\\GNSS'

In [7]:
# make output filename from input file automatically

# out_file="2021-07-05_10_GNRMC_out_test.csv"
# out_file="VRS直_2021-07-05-12-48-07_get.rtk"
# out_file=base_name+"_get.rtk"
out_file=base_name+"_GSI.in"
out_file
# out_file="VRS直_2021-07-05-12-48-07_get.rtk"

'C:\\Users\\survey\\Desktop\\nmea20210705\\2021-07-05_09+10_GNRMC_GSI.in'

## nmeaはUTF-8
## Trimbleはshift_jisなはず

In [9]:
enc_sty="UTF-8"

# enc_sty="shift_jis"

In [151]:
# with open("2021-07-05_10.nmea", "r", encoding="UTF-8", errors="", newline="" ) as f:
# with open(inp_file, "r", encoding="UTF-8", errors="", newline="" ) as f:
with open(inp_file, "r", encoding=enc_sty, errors="", newline="" ) as f:
    lst = csv.reader(f, delimiter=",")
    df = pd.DataFrame(lst).rename(columns={0:'ID',1:'time',2:'de',3:'lat',5:'lon',7:'knot',9:'date',12:'mode'})
    # df = pd.DataFrame(lst)

get_id='GNRMC'

In [153]:
df_gnrmc=df.query('ID in ["$GNRMC"]')
# df_gnrmc['lat']=df_gnrmc['lat'].astype(float)

In [115]:
df_gnrmc.dtypes

ID      object
time    object
de      object
lat     object
4       object
lon     object
6       object
knot    object
8       object
date    object
10      object
11      object
mode    object
13      object
14      object
15      object
16      object
17      object
18      object
19      object
20      object
21      object
22      object
23      object
24      object
25      object
26      object
27      object
28      object
29      object
30      object
31      object
32      object
33      object
34      object
35      object
36      object
37      object
38      object
39      object
40      object
41      object
42      object
43      object
44      object
45      object
46      object
47      object
48      object
49      object
dtype: object

In [157]:
df_gnrmc

,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,000145.00,V,,,,,,,050721,...,,,,,,,,,,
1,$GNRMC,000145.12,V,,,,,,,050721,...,,,,,,,,,,
2,$GNRMC,000145.25,V,,,,,,,050721,...,,,,,,,,,,
3,$GNRMC,000145.37,V,,,,,,,050721,...,,,,,,,,,,
4,$GNRMC,000145.50,V,,,,,,,050721,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,010834.38,A,3511.0129030,N,13707.1113436,E,0.015,,050721,...,,,,,,,,,,
30715,$GNRMC,010834.50,A,3511.0129025,N,13707.1113439,E,0.024,,050721,...,,,,,,,,,,
30716,$GNRMC,010834.63,A,3511.0129041,N,13707.1113448,E,0.015,,050721,...,,,,,,,,,,
30717,$GNRMC,010834.75,A,3511.0129040,N,13707.1113448,E,0.025,,050721,...,,,,,,,,,,


In [155]:
pd.options.display.float_format = '{:.7f}'.format

In [158]:
# df_gnrmc['lat'] == ''] = None

df_gnrmc=df_gnrmc.replace('', np.nan)

In [159]:
df_gnrmc

,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,000145.00,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$GNRMC,000145.12,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$GNRMC,000145.25,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$GNRMC,000145.37,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$GNRMC,000145.50,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,010834.38,A,3511.0129030,N,13707.1113436,E,0.015,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30715,$GNRMC,010834.50,A,3511.0129025,N,13707.1113439,E,0.024,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30716,$GNRMC,010834.63,A,3511.0129041,N,13707.1113448,E,0.015,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30717,$GNRMC,010834.75,A,3511.0129040,N,13707.1113448,E,0.025,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
df_gnrmc['time']=df_gnrmc['time'].astype(float)



In [147]:
df_gnrmc

,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,145.00,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$GNRMC,145.12,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$GNRMC,145.25,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$GNRMC,145.37,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$GNRMC,145.50,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,10834.38,A,3511.0129030,N,13707.1113436,E,0.015,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30715,$GNRMC,10834.50,A,3511.0129025,N,13707.1113439,E,0.024,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30716,$GNRMC,10834.63,A,3511.0129041,N,13707.1113448,E,0.015,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30717,$GNRMC,10834.75,A,3511.0129040,N,13707.1113448,E,0.025,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
df_gnrmc['lat']=df_gnrmc['lat'].astype(float)

In [162]:
df_gnrmc

,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,000145.00,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$GNRMC,000145.12,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$GNRMC,000145.25,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$GNRMC,000145.37,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$GNRMC,000145.50,V,NaN,NaN,NaN,NaN,NaN,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,010834.38,A,3511.0129030,N,13707.1113436,E,0.015,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30715,$GNRMC,010834.50,A,3511.0129025,N,13707.1113439,E,0.024,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30716,$GNRMC,010834.63,A,3511.0129041,N,13707.1113448,E,0.015,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30717,$GNRMC,010834.75,A,3511.0129040,N,13707.1113448,E,0.025,NaN,050721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
df_gnrmc.dtypes



ID       object
time     object
de       object
lat     float64
4        object
lon      object
6        object
knot     object
8        object
date     object
10      float64
11      float64
mode     object
13       object
14      float64
15      float64
16      float64
17      float64
18      float64
19      float64
20      float64
21      float64
22      float64
23      float64
24      float64
25      float64
26      float64
27      float64
28      float64
29      float64
30      float64
31      float64
32      float64
33      float64
34      float64
35      float64
36      float64
37      float64
38      float64
39      float64
40      float64
41      float64
42      float64
43      float64
44      float64
45      float64
46      float64
47      float64
48      float64
49      float64
dtype: object

df_gnrmc['lat'] = pd.to_numeric(df_gnrmc['lat'], errors='coerce', downcast='float')
# df_gnrmc['lat']=df_gnrmc['lat'].astype(float)

df_gnrmc['lat'] = pd.to_numeric(df_gnrmc['lat'], errors='coerce', downcast='float')

In [110]:
df_gnrmc

,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,145.00,V,NaN,,,,,,050721,...,,,,,,,,,,
1,$GNRMC,145.12,V,NaN,,,,,,050721,...,,,,,,,,,,
2,$GNRMC,145.25,V,NaN,,,,,,050721,...,,,,,,,,,,
3,$GNRMC,145.37,V,NaN,,,,,,050721,...,,,,,,,,,,
4,$GNRMC,145.50,V,NaN,,,,,,050721,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,10834.38,A,3511.012939,N,13707.1113436,E,0.015,,050721,...,,,,,,,,,,
30715,$GNRMC,10834.50,A,3511.012939,N,13707.1113439,E,0.024,,050721,...,,,,,,,,,,
30716,$GNRMC,10834.63,A,3511.012939,N,13707.1113448,E,0.015,,050721,...,,,,,,,,,,
30717,$GNRMC,10834.75,A,3511.012939,N,13707.1113448,E,0.025,,050721,...,,,,,,,,,,


In [112]:
df_gnrmc.dtypes

ID       object
time    float64
de       object
lat     float32
4        object
lon      object
6        object
knot     object
8        object
date     object
10       object
11       object
mode     object
13       object
14       object
15       object
16       object
17       object
18       object
19       object
20       object
21       object
22       object
23       object
24       object
25       object
26       object
27       object
28       object
29       object
30       object
31       object
32       object
33       object
34       object
35       object
36       object
37       object
38       object
39       object
40       object
41       object
42       object
43       object
44       object
45       object
46       object
47       object
48       object
49       object
dtype: object

In [86]:
df_gnrmc_ll=df_gnrmc.filter(items=['lat','lon'])
df_gnrmc_ll

,lat,lon
0,,
1,,
2,,
3,,
4,,
...,...,...
30714,3511.0129030,13707.1113436
30715,3511.0129025,13707.1113439
30716,3511.0129041,13707.1113448
30717,3511.0129040,13707.1113448


In [95]:
df_gnrmc_ll.dtypes

lat    object
lon    object
dtype: object

In [105]:
df_gnrmc_ll_f=df_gnrmc_ll.astype(float)

ValueError: could not convert string to float: ''

In [84]:
df_gnrmc_ll['lat']=df_gnrmc_ll['lat'].astype(float)
# df_gnrmc_ll['lon']=df_gnrmc_ll['lon'].astype(float)
# df_gnrmc_ll['lon']=pd.to_numeric(df_gnrmc_ll['lon'])

ValueError: could not convert string to float: ''

In [72]:
df_gnrmc_ll

,lat,lon
0,NaN,
1,NaN,
2,NaN,
3,NaN,
4,NaN,
...,...,...
30714,3511.012903,13707.1113436
30715,3511.012903,13707.1113439
30716,3511.012904,13707.1113448
30717,3511.012904,13707.1113448


In [59]:
df_gnrmc_ll = df_gnrmc_ll.reset_index(drop=True)
df_gnrmc_ll
type(df_gnrmc_ll)

pandas.core.frame.DataFrame

In [61]:
df_gnrmc_ll.to_numpy()

array([['', ''],
       ['', ''],
       ['', ''],
       ...,
       ['3511.0129041', '13707.1113448'],
       ['3511.0129040', '13707.1113448'],
       ['3511.0129026', '13707.1113445']], dtype=object)

In [62]:
df_gnrmc_ll.astype(float64)

NameError: name 'float64' is not defined

In [53]:
df_gnrmc_ll=df_gnrmc_ll.values

In [55]:
type(df_gnrmc_ll)
print(df_gnrmc_ll.dtype)

object


df_gnrmc_ll[['lat','lon']]=df_gnrmc_ll[['lat','lon']].astype(float)

In [22]:
type(df_gnrmc_ll)

NameError: name 'astype' is not defined

In [11]:
df



,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,000145.00,V,,,,,,,050721,...,,,,,,,,,,
1,$GNRMC,000145.12,V,,,,,,,050721,...,,,,,,,,,,
2,$GNRMC,000145.25,V,,,,,,,050721,...,,,,,,,,,,
3,$GNRMC,000145.37,V,,,,,,,050721,...,,,,,,,,,,
4,$GNRMC,000145.50,V,,,,,,,050721,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,010834.38,A,3511.0129030,N,13707.1113436,E,0.015,,050721,...,,,,,,,,,,
30715,$GNRMC,010834.50,A,3511.0129025,N,13707.1113439,E,0.024,,050721,...,,,,,,,,,,
30716,$GNRMC,010834.63,A,3511.0129041,N,13707.1113448,E,0.015,,050721,...,,,,,,,,,,
30717,$GNRMC,010834.75,A,3511.0129040,N,13707.1113448,E,0.025,,050721,...,,,,,,,,,,


In [13]:
df_latlon=df.values

In [15]:
df_laton=df_latlon.filter(items=['lat','lon'])
df_latlon
# df_gnrmc_ll

AttributeError: 'numpy.ndarray' object has no attribute 'filter'

In [14]:
df

,ID,time,de,lat,4,lon,6,knot,8,date,...,40,41,42,43,44,45,46,47,48,49
0,$GNRMC,000145.00,V,,,,,,,050721,...,,,,,,,,,,
1,$GNRMC,000145.12,V,,,,,,,050721,...,,,,,,,,,,
2,$GNRMC,000145.25,V,,,,,,,050721,...,,,,,,,,,,
3,$GNRMC,000145.37,V,,,,,,,050721,...,,,,,,,,,,
4,$GNRMC,000145.50,V,,,,,,,050721,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30714,$GNRMC,010834.38,A,3511.0129030,N,13707.1113436,E,0.015,,050721,...,,,,,,,,,,
30715,$GNRMC,010834.50,A,3511.0129025,N,13707.1113439,E,0.024,,050721,...,,,,,,,,,,
30716,$GNRMC,010834.63,A,3511.0129041,N,13707.1113448,E,0.015,,050721,...,,,,,,,,,,
30717,$GNRMC,010834.75,A,3511.0129040,N,13707.1113448,E,0.025,,050721,...,,,,,,,,,,


# ここって合ってる？
df_gnrmc_ll[['lat','lon']]=df_gnrmc_ll[['lat','lon']].astype(float)

# ここでnumpy化します

In [36]:
df_gnrmc_ll_np=df_gnrmc_ll.values
df_gnrmc_ll_np

array([['', ''],
       ['', ''],
       ['', ''],
       ...,
       ['3511.0129041', '13707.1113448'],
       ['3511.0129040', '13707.1113448'],
       ['3511.0129026', '13707.1113445']], dtype=object)

In [43]:
type(df_gnrmc_ll_np)

numpy.ndarray

In [44]:
df_gnrmc_ll_np

array([['', ''],
       ['', ''],
       ['', ''],
       ...,
       ['3511.0129041', '13707.1113448'],
       ['3511.0129040', '13707.1113448'],
       ['3511.0129026', '13707.1113445']], dtype=object)

In [45]:
df_gnrmc_ll_np.astype(float)

ValueError: could not convert string to float: ''

In [39]:
# df_gnrmc_ll_np_f=df_gnrmc_ll_np.astype(np.float)
df_gnrmc_ll_np_f=np.asfarray(df_gnrmc_ll_np,dtype=float)

ValueError: could not convert string to float: ''

df_gnrmc_ll_np[['lat','lon']]=df_gnrmc_ll_np[['lat','lon']].astype(float)



In [40]:
type(df_gnrmc_ll_np)
# df_gnrmc_ls

numpy.ndarray

In [42]:
# print(df_gnrmc_ll_np.dtype)
df_gnrmc_ll_np.dtype

dtype('O')

a=10
a
type(a)

In [31]:
a=10

df_gnrmc_ll_np + a

TypeError: can only concatenate str (not "int") to str

lat_int=df_gnrmc_ls[:,1])

In [ ]:
# df.query('cols in ["GG1","GG2"]')
# df.query('ID in ["GG1","GG2"]').to_csv(out_file,header=False, index=False)
df.query('ID in ["$GNRMC"]').to_csv(out_file,header=False, index=False)
# df[df[0].str.contains(id)].to_csv(out_file,header=False, index=False)

FIN
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# 灯台下暗し
# 欲しい列だけ指定
df_gnrmc=df_gnrmc.drop(df_gnrmc.columns[[0,1,2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]], axis=1)
# concatenated_df=concatenated_df.drop(concatenated_df.columns[[0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,20,21,22,23,24,25,26,27,28,29,30,31]], axis=1)


In [ ]:
# concatenate horizontally

# id='GNRMC'
id="GG1|GG2"


df

# oi=df.iloc[0]
# print(oi)
oi2=df.iloc[:,1]
print(oi2)

# print(df[0].str.contains('$GNRMC'))
# df[0].str.contains('GNRMC')
df[0].str.contains(id)



# df[df[0].str.contains('GNRMC')].to_csv("2021-07-05_10_GNRMC.csv",header=False, index=False)
df[df[0].str.contains(id)].to_csv(out_file,header=False, index=False)

##################################################################
#################################
FIN

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# df = pd.read_csv('2021-07-04_16.csv', index_col=0)
# df = pd.read_csv('2021-07-04_16.csv')
df = pd.read_csv('2021-07-04_16.csv', header=None)
print(df)

csvfile = '2021-07-05_07.nmea'
f = open(csvfile, "r")
reader = csv.reader(f)
# header = next(reader)

for row in reader:
        print(row)

f.close()


# df[0].isin([GNRMC])


df.to_csv("GNRMMC_1.csv")

type(kore)

print(df[df[0].str.contains('$GNRMC')])

df.iloc[5]

df.loc["$GNRMC"]


